# Experiment 02

This Experiment consists in reproduce the exactly the same experiment as proposed by Martinez (https://www.researchgate.net/publication/221533996_From_an_artificial_neural_network_to_a_stock_market_day-trading_system_A_case_study_on_the_BMF_BOVESPA).

In Summary:
- Each node from ANN Implements a Logistic Function.
- 3 Classic layers:
    - Until 33 Nodes in the Input Layer
    - 2 Node as Outpuy Layers
    - The hidden layers has sqrt(# Input nodes * # Output Nodes) nodes
- 100,000 epochs
- Learning Rate constat and equals to 0.01
- The Momentum from the Back-Propagation Algorithm is 0.8


In [1]:
import numpy as np
import pandas as pd

import Utils 
from Utils import calculateEMA, calculateBB, addPreviousDays, removeColumns, getPeriod, mape, mae, createDate, getDate, printResult

import matplotlib.pyplot as plt
import matplotlib.dates as mpdates
import mplfinance as mpf
from mplfinance.original_flavor import candlestick_ohlc

import warnings
warnings.filterwarnings('ignore')

from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score

### Constants and Parameters

In [2]:
dfColumns = ['date','open', 'high', 'low','close','tickvol','volume','spread']
newColumns = ['open', 'high', 'low', 'close']
col15 = ['open','d-5_high','d-5_low','d-4_high','d-4_low','d-3_high','d-3_low','d-2_high','d-2_low','d-1_high','d-1_low','EMA_high','EMA_low','UB_close','LB_close']
col25 = ['open','d-5_open','d-5_high','d-5_low','d-5_close','d-4_open','d-4_high','d-4_low','d-4_close','d-3_open','d-3_high','d-3_low','d-3_close','d-2_open','d-2_high','d-2_low','d-2_close','d-1_open','d-1_high','d-1_low','d-1_close','EMA_high' ,'EMA_low','UB_close','LB_close']
DAYS = 5
TEST_BEGIN = '2021.04.06'
TEST_END = '2021.06.01'
TRAIN_BEGIN = '2020.03.31'
TRAIN_END = '2021.05.31'
TIMES = 10

### Auxiliary functions

In [3]:
def Normalize(df):
    date = df['date']
    maxVal = df[['high', 'low']].max()
    normalized = removeColumns(df,['date'])/(removeColumns(df,['date']).max()*2)
    
    
    df.loc[:, df.columns != 'date'] = normalized
    
    return df, maxVal

### Opening the dataset

In [4]:
df = pd.read_csv('../Data/PETR4_Daily.csv', sep = "\t")
df.columns = dfColumns
df = removeColumns(df, ['tickvol', 'volume','spread'])

df2 = pd.read_csv('../Data/VALE3_Daily.csv', sep = "\t")

df2.columns = dfColumns
df2 = removeColumns(df2, ['tickvol', 'volume','spread'])

df.tail(290)

,date,open,high,low,close
1049,2020.03.31,13.16,14.07,13.16,13.54
1050,2020.04.01,13.01,14.08,12.89,13.84
1051,2020.04.02,14.90,16.01,14.52,15.01
1052,2020.04.03,15.77,15.83,14.44,14.84
1053,2020.04.06,15.53,15.58,14.69,15.26
...,...,...,...,...,...
1334,2021.05.26,25.91,26.18,25.65,26.09
1335,2021.05.27,26.08,26.25,25.77,25.91
1336,2021.05.28,26.20,27.02,26.15,26.99
1337,2021.05.31,26.96,27.20,26.78,26.87


In [5]:
#PETR4
df = calculateEMA(df,DAYS,newColumns)
df = calculateBB(df,DAYS,newColumns)
df = addPreviousDays(df,DAYS, newColumns)

#VALE3
df2 = calculateEMA(df2,DAYS,newColumns)
df2 = calculateBB(df2,DAYS,newColumns)
df2 = addPreviousDays(df2,DAYS, newColumns)

In [6]:
##df = getPeriod(df, '2020.03.02','2021.04.30', True)
##initialDf = getPeriod(df, '2020.03.02','2021.04.30', True)

In [7]:
initialDf_PETR4 = getPeriod(df, '2019.01.02','2021.06.01', True)
initialDf_VALE3 = getPeriod(df2, '2020.01.02','2021.06.01', True)

In [8]:
initialDf_PETR4

,date,open,high,low,close,EMA_open,EMA_high,EMA_low,EMA_close,UB_open,...,d-3_low,d-3_close,d-4_open,d-4_high,d-4_low,d-4_close,d-5_open,d-5_high,d-5_low,d-5_close
0,2019.01.02,21.18,22.73,20.93,22.60,20.609175,21.499077,20.380957,21.271656,21.776315,...,19.18,20.37,19.55,20.08,19.41,19.69,20.38,20.39,19.50,19.63
1,2019.01.03,22.51,23.32,22.36,23.16,21.242783,22.106051,21.040638,21.901104,23.157151,...,19.99,20.36,19.34,20.37,19.18,20.37,19.55,20.08,19.41,19.69
2,2019.01.04,23.35,23.43,22.99,23.22,21.945189,22.547367,21.690425,22.340736,24.186721,...,20.74,21.31,20.22,20.65,19.99,20.36,19.34,20.37,19.18,20.37
3,2019.01.07,23.35,24.35,23.20,23.59,22.413459,23.148245,22.193617,22.757157,24.644733,...,20.93,22.60,20.77,21.45,20.74,21.31,20.22,20.65,19.99,20.36
4,2019.01.08,23.86,23.88,23.27,23.45,22.895640,23.392163,22.552411,22.988105,24.953473,...,22.36,23.16,21.18,22.73,20.93,22.60,20.77,21.45,20.74,21.31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
594,2021.05.26,25.91,26.18,25.65,26.09,26.114791,26.354219,25.760657,26.042263,26.608719,...,25.94,25.95,26.15,26.28,25.72,25.93,25.85,26.35,25.80,26.15
595,2021.05.27,26.08,26.25,25.77,25.91,26.103194,26.319479,25.763771,25.998175,26.604087,...,25.99,26.39,26.08,26.31,25.94,25.95,26.15,26.28,25.72,25.93
596,2021.05.28,26.20,27.02,26.15,26.99,26.135462,26.552986,25.892514,26.328784,26.617551,...,25.78,25.84,26.28,26.52,25.99,26.39,26.08,26.31,25.94,25.95
597,2021.05.31,26.96,27.20,26.78,26.87,26.410308,26.768657,26.188343,26.509189,27.147657,...,25.65,26.09,26.48,26.62,25.78,25.84,26.28,26.52,25.99,26.39


In [9]:
initialDf_VALE3

,date,open,high,low,close,EMA_open,EMA_high,EMA_low,EMA_close,UB_open,...,d-3_low,d-3_close,d-4_open,d-4_high,d-4_low,d-4_close,d-5_open,d-5_high,d-5_low,d-5_close
0,2020.01.02,50.52,50.95,50.31,50.85,50.225554,50.487454,49.864452,50.203124,50.685390,...,49.90,50.16,50.26,50.33,49.80,49.96,50.14,50.34,49.57,50.16
1,2020.01.03,50.24,51.01,50.15,50.47,50.230370,50.661636,49.959634,50.292083,50.686307,...,50.00,50.16,50.17,50.35,49.90,50.16,50.26,50.33,49.80,49.96
2,2020.01.06,50.35,50.35,49.74,50.17,50.270246,50.557757,49.886423,50.251388,50.671662,...,49.79,49.88,50.53,50.54,50.00,50.16,50.17,50.35,49.90,50.16
3,2020.01.07,50.07,50.72,49.98,50.54,50.203498,50.611838,49.917615,50.347592,50.614274,...,50.31,50.85,50.21,50.41,49.79,49.88,50.53,50.54,50.00,50.16
4,2020.01.08,50.73,50.89,50.18,50.55,50.378998,50.704559,50.005077,50.415062,50.890606,...,50.15,50.47,50.52,50.95,50.31,50.85,50.21,50.41,49.79,49.88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
346,2021.05.26,106.37,110.32,106.03,110.20,109.166674,111.131617,107.622606,109.807900,113.973861,...,108.50,109.40,111.70,112.10,110.20,111.11,112.10,112.78,109.84,112.25
347,2021.05.27,110.80,112.09,109.77,110.99,109.711116,111.451078,108.338404,110.201933,113.405981,...,107.51,109.78,111.11,111.25,108.50,109.40,111.70,112.10,110.20,111.11
348,2021.05.28,111.11,112.75,109.42,111.59,110.177411,111.884052,108.698936,110.664622,113.405981,...,106.78,107.05,108.00,110.14,107.51,109.78,111.11,111.25,108.50,109.40
349,2021.05.31,112.50,114.89,112.50,114.78,110.951607,112.886035,109.965957,112.036415,114.811204,...,106.03,110.20,110.28,110.89,106.78,107.05,108.00,110.14,107.51,109.78


#### Experimento 02: Reproducing the Experiment proposed by Martinez

In [10]:
def exp02(df, stock, inputColumns = []):
    aux, maxVal = Normalize(df)
    df = aux
    
    df_train = getPeriod(df, TRAIN_BEGIN, TRAIN_END)
    df_test = getPeriod(df, TEST_BEGIN, TEST_END)
    
    
    if(inputColumns == []):
        x_train = df_train.drop(['date', 'high', 'low', 'close'], axis=1)
        y_train = df_train[['high','low']]

        x_test = df_test.drop(['date', 'high', 'low', 'close'], axis=1)
        y_test = df_test[['high','low']]
    else:
        x_train = df_train[inputColumns]
        y_train = df_train[['high','low']]

        x_test = df_test[inputColumns]
        y_test = df_test[['high','low']]
    
    MLP = MLPRegressor(hidden_layer_sizes = (8), #sqrt(33*2) according to the article
                   momentum = 0.8,
                   max_iter = 100000,
                   activation = 'logistic',
                   learning_rate = 'constant',
                   learning_rate_init = 0.01)

    MLP.fit(x_train, y_train)
    y_pred = MLP.predict(x_test)
    
    y_predFix = y_pred
    y_predFix[:,0] = y_predFix[:,0]*maxVal['high']*2
    y_predFix[:,1] = y_predFix[:,1]*maxVal['low']*2

    y_testFix = y_test*maxVal*2
    
    #print(stock)
    
    errorMax = mape(y_testFix['high'],y_predFix[:,0])
    errorMin = mape(y_testFix['low'],y_predFix[:,1])
    ##printResult(mape(y_testFix['high'],y_predFix[:,0]), 'High')
    ##printResult(mape(y_testFix['low'],y_predFix[:,1]), 'Low')
    #print (mae(y_testFix['high'],y_predFix[:,0]))
    #print (mae(y_testFix['low'],y_predFix[:,1]))
    
    return errorMax, errorMin

def test30timesExp02(df,stock, phrase, columnInput = []):
    errorMax = []
    errorMin = []
    
    for i in range(1, TIMES+1):
        if(columnInput == []):
            errorMax_petr4, errorMin_petr4 = exp02(df, stock)
        else: 
            errorMax_petr4, errorMin_petr4 = exp02(df, stock, columnInput)
        
        errorMax.append(errorMax_petr4[0])
        errorMin.append(errorMin_petr4[0])
    
    errorMax = np.array(errorMax)
    errorMin = np.array(errorMin)
    print(phrase)
    printResult((errorMax.mean(),errorMax.std()), 'High')
    printResult((errorMin.mean(),errorMin.std()), 'Low')
    
test30timesExp02(initialDf_PETR4, 'PETR4', 'PETR4 15 Variables', col15) 
test30timesExp02(initialDf_PETR4, 'PETR4', 'PETR4 25 Variables', col25) 
test30timesExp02(initialDf_PETR4, 'PETR4', 'PETR4 33 Variables') 
print('\n')
test30timesExp02(initialDf_VALE3, 'VALE3', 'VALE3 15 Variables', col15) 
test30timesExp02(initialDf_VALE3, 'VALE3', 'VALE3 25 Variables', col25) 
test30timesExp02(initialDf_VALE3, 'VALE3', 'VALE3 33 Variables') 

PETR4 15 Variables
High: 8.03 +- 3.4 %
Low: 6.73 +- 2.61 %
PETR4 25 Variables
High: 6.99 +- 1.69 %
Low: 5.44 +- 2.04 %
PETR4 33 Variables
High: 7.89 +- 5.41 %
Low: 7.57 +- 3.63 %


VALE3 15 Variables
High: 10.93 +- 5.35 %
Low: 9.97 +- 4.64 %
VALE3 25 Variables
High: 9.14 +- 3.76 %
Low: 14.89 +- 8.61 %
VALE3 33 Variables
High: 8.04 +- 3.65 %
Low: 6.42 +- 3.91 %
